In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import math
import os
import time

In [3]:
# get all train texts
dir = "../"
train_texts = ""
files = os.listdir(dir + "shahname/train")
for path in files:
	train_texts += open(dir + "shahname/train/" + path, 'rb').read().decode(encoding='utf-8')

In [96]:
file_text = open(dir + "allShahnameWords.txt", 'rb').read().decode(encoding = "utf-8")
vocab = file_text.split("\n")

# vocab = sorted(set(file_text))
vocab.append(" ")
# vocab.append("\n")

ids_from_words = preprocessing.StringLookup(
    vocabulary=vocab, mask_token=None)
# print(vocab[20])
# a= ids_from_words("اباپیل")
# a

In [97]:
tf.strings.split(train_texts)[50].numpy().decode("utf-8")

'کاخ'

In [98]:
# turn ids into human readable text

words_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(words_from_ids(ids))#, axis=-1)


In [99]:
all_ids = ids_from_words(tf.strings.split(train_texts))#tf.strings.unicode_split(train_texts, 'UTF-8'))
all_ids

<tf.Tensor: shape=(533516,), dtype=int64, numpy=array([ 8494, 10084,   390, ...,  8494, 11220,  6646])>

In [100]:
# all_ids[0].numpy()
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(words_from_ids(ids).numpy().decode('utf-8'))

چو
دشت
از
گیا
گشت
چون
پرنیان
ببستند
گردان
توران


In [101]:
seq_length = 100
examples_per_epoch = len(train_texts)//(seq_length+1)

In [102]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  tmp = words_from_ids(seq).numpy()
  print(''.join([str(s, encoding='UTF-8') for s in tmp]))
  # print(chars_from_ids(seq))

چودشتازگیاگشتچونپرنیانببستندگردانتورانمیانسپاهیبیامدزترکانوچینهمازگرزدارانخاورزمینکهآنرامیانوکرانهنبودهمانبختنوذرجوانهنبودچولشکربهنزدیکجیحونرسیدخبرنزدپورفریدونرسیدسپاهجهانداربیرونشدندزکاخهمایونبههامونشدندبهراهدهستاننهادندرویسپهدارشانقارنرزم‌جویشهنشاهنوذرپسپشتاویجهانیسراسرپرازگفتوگویچولشکربهپیشدهستانرسیدتوگفتیکهخورشیدشدناپدیدسراپردهٔنوذرشهریارکشیدندبردشتپیشحصارخوداندردهستاننیاراستجنگبرین


In [103]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy().decode("utf-8"))

چودشتازگیاگشتچونپرنیانببستندگردانتورانمیانسپاهیبیامدزترکانوچینهمازگرزدارانخاورزمینکهآنرامیانوکرانهنبودهمانبختنوذرجوانهنبودچولشکربهنزدیکجیحونرسیدخبرنزدپورفریدونرسیدسپاهجهانداربیرونشدندزکاخهمایونبههامونشدندبهراهدهستاننهادندرویسپهدارشانقارنرزم‌جویشهنشاهنوذرپسپشتاویجهانیسراسرپرازگفتوگویچولشکربهپیشدهستانرسیدتوگفتیکهخورشیدشدناپدیدسراپردهٔنوذرشهریارکشیدندبردشتپیشحصارخوداندردهستاننیاراستجنگبرین
برنیامدزمانیدرنگکهافراسیاباندرایرانزمیندوسالارکردازبزرگانگزینشماساسودیگرخزروانگردزلشکرسوارانبدیشانسپردزجنگآورانمردچونسیهزاربرفتندشایستهٔکارزارسویزابلستاننهادندرویزکینهبهدستاننهادندرویخبرشدکهسامنریمانبمردهمیدخمهسازدورازالگردازانسختشادانشدافراسیاببدیدآنکهبختاندرآمدبهخواببیامدچوپیشدهستانرسیدبرابرسراپرده‌ایبرکشیدسپهراکهدانستکردنشماربروچارصدباربشمرهزاربجوشیدگفتیهمهریگوشخبیابانسراسرچوموروملخابا
شاهنوذرصدوچلهزارهماناکهبودندجنگیسواربهلشکرنگهکردافراسیابهیونیبرافگندهنگامخوابیکینامهبنوشتسویپشنگکهجستیمنیکیوآمدبهچنگهمهلشکرنوذراربشکریمشکارندودرزیرپیبسپریمدگرسامرفتازدرشهریارهمانانیایدبدینکارزارستودانهمیسازدشزالزرندارد

In [104]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


dataset = sequences.map(split_input_target)

In [105]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy().decode("utf-8"))
    print("Target:", text_from_ids(target_example).numpy().decode("utf-8"))

Input : چودشتازگیاگشتچونپرنیانببستندگردانتورانمیانسپاهیبیامدزترکانوچینهمازگرزدارانخاورزمینکهآنرامیانوکرانهنبودهمانبختنوذرجوانهنبودچولشکربهنزدیکجیحونرسیدخبرنزدپورفریدونرسیدسپاهجهانداربیرونشدندزکاخهمایونبههامونشدندبهراهدهستاننهادندرویسپهدارشانقارنرزم‌جویشهنشاهنوذرپسپشتاویجهانیسراسرپرازگفتوگویچولشکربهپیشدهستانرسیدتوگفتیکهخورشیدشدناپدیدسراپردهٔنوذرشهریارکشیدندبردشتپیشحصارخوداندردهستاننیاراستجنگ
Target: دشتازگیاگشتچونپرنیانببستندگردانتورانمیانسپاهیبیامدزترکانوچینهمازگرزدارانخاورزمینکهآنرامیانوکرانهنبودهمانبختنوذرجوانهنبودچولشکربهنزدیکجیحونرسیدخبرنزدپورفریدونرسیدسپاهجهانداربیرونشدندزکاخهمایونبههامونشدندبهراهدهستاننهادندرویسپهدارشانقارنرزم‌جویشهنشاهنوذرپسپشتاویجهانیسراسرپرازگفتوگویچولشکربهپیشدهستانرسیدتوگفتیکهخورشیدشدناپدیدسراپردهٔنوذرشهریارکشیدندبردشتپیشحصارخوداندردهستاننیاراستجنگبرین


In [106]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [107]:
# Model
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.LSTM(rnn_units,
                                    #stateful=True 
                                   return_sequences=True,
                                   return_state=True
    )
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      h_state, c_state = self.gru.get_initial_state(x)
    else:
      h_state, c_state = states
    #x, states = self.gru(x, initial_state=states, training=training)
    x, h_state,c_state = self.gru(x, initial_state=[h_state,c_state], training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, h_state, c_state
    else:
      return x

In [108]:
vocab_size

19777

In [109]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_words.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [110]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 19778) # (batch_size, sequence_length, vocab_size)


In [111]:
model.summary()

Model: "my_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  5063168   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  5246976   
_________________________________________________________________
dense_3 (Dense)              multiple                  20272450  
Total params: 30,582,594
Trainable params: 30,582,594
Non-trainable params: 0
_________________________________________________________________


In [112]:
# Befor training
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 19778)  # (batch_size, sequence_length, vocab_size)
Mean loss:         9.892313


In [113]:
tf.exp(mean_loss).numpy()

19777.752

In [114]:
model.compile(optimizer='adam', loss=loss)

## Training

In [89]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/30
82/82 [==============================] - 28s 320ms/step - loss: 7.6073
Epoch 2/30
82/82 [==============================] - 28s 334ms/step - loss: 7.2250
Epoch 3/30
82/82 [==============================] - 28s 333ms/step - loss: 7.1333
Epoch 4/30
82/82 [==============================] - 27s 328ms/step - loss: 6.9167
Epoch 5/30
82/82 [==============================] - 28s 336ms/step - loss: 6.7314
Epoch 6/30
82/82 [==============================] - 29s 339ms/step - loss: 6.5451
Epoch 7/30
82/82 [==============================] - 28s 335ms/step - loss: 6.3184
Epoch 8/30
82/82 [==============================] - 28s 332ms/step - loss: 6.0864
Epoch 9/30
82/82 [==============================] - 28s 339ms/step - loss: 5.8659
Epoch 10/30
82/82 [==============================] - 29s 341ms/step - loss: 5.6563
Epoch 11/30
82/82 [==============================] - 28s 335ms/step - loss: 5.4638
Epoch 12/30
82/82 [==============================] - 28s 334ms/step - loss: 5.2962
Epoch 13/30
8

In [115]:
# Save model
save_dir = "model-word-v1.h5"
# model.save_weights(save_dir)  # to store

# Load model
model.load_weights(save_dir)  # to load

In [116]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # h_state, c_state = states
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, h_state, c_state = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, h_state, c_state

In [117]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

print("Input:\n", text_from_ids(input_example_batch[0]).numpy().decode("utf-8"))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy().decode("utf-8"))

Input:
 ندیدبرونامشاهیواوناپدیدصدمسالروزیبهدریایچینپدیدآمدآنشاهناپاکدیننهانگشتهبودازبداژدهانیامدبهفرجامهمزورهاچوضحاکشآوردناگهبهچنگیکایکندادشزمانیدرنگبهارشسراسربهدونیمکردجهانراازوپاکبی‌بیمکردشدآنتختشاهیوآندستگاهزمانهربودشچوبیجادهکاهازوبیشبرتختشاهیکهبودبرانرنجبردنچهآمدشسودگذشتهبروسالیانهفتصدپدیدآوریدههمهنیکوبدچهبایدهمهزندگانیدرازچوگیتینخواهدگشادنترازهمی

Next Char Predictions:
 چمیدنشرابسروربرخواربوسخورهٔبنوککمترمپیروزه،نفتنوش‌لبغرندهجهانبانقیصرمستبرستپایانبپرسندهوبستهوپیوندپرده‌داراسپیبپساورامخوانیدشتابنده‌اندوبنشاهنشاهانمگرشانپیلبازیمردارچارپایانوبشادینیابیمچاکریگوهرشناسلشکری‌دارچنگقندگریختبی‌بارسپاربدانچتشهمزسیدادییمگذاردادخواهندگانومغزیخرمپیداستوبرآسیمهبی‌غمبرداشتیدشارستانگزینندورازاداندرونشآسانیبکشتمپیل‌بازیوکامجویستکهترایارایزکسریبخوابیدکنابدبرکمسرانجمنیالگشودافگندپولادبرزشیربرخواندمبامشآبگیریشهشبخوربیدنشناختندویرانیراگشادنخرامیدمبخورشیدوماهپای‌بندفرودینازنوشبدکمنشببارغرمکرده‌ایدگرایندجسمبیرنجرزم‌زنخواندتدرندهمرزمدرگرفتازلشکری


In [118]:
one_step_model = OneStep(model, words_from_ids, ids_from_words)

## Generate fake texts

In [140]:
start = time.time()
next_char = tf.constant(['مرا گفت کاین نامهٔ شهریار'])
states = None
zero_out = tf.keras.layers.Lambda(lambda x: tf.zeros_like(x))(next_char)
result = [next_char]

for n in range(30):
  next_char, h_state, c_state = one_step_model.generate_one_step(next_char, states=states)
  states = [h_state, c_state]
  result.append(next_char)

result = tf.strings.join(result," ")
end = time.time()
print(result[0].numpy().decode('utf-8'))
print('\nRun time:', end - start)

مرا گفت کاین نامهٔ شهریار نیست بگفت پاک باشد مرد گر بود گر شد مرد که پاک باشد ما نداریم بود بپیش پاک داری نگار جهان پاک باید اژدهاست آفریننده‌ام پاک هست پاک فرمان پاک

Run time: 0.11357998847961426


## Generated texts

> مرا گفت کاین نامهٔ شهریار

> نیست بگفت پاک باشد مرد گر بود

>  گر شد مرد که پاک باشد ما نداریم 

> بود بپیش پاک داری نگار جهان پاک
